The code in this notebook converts a subset of NOAA geopolar data that is stored locally on the department server from big-in-space chunks to big-in-time chunks. Both datasub sets are full space with 124 time steps. It uses the rechunker library to do the conversion.

(I believe this was run with a local rechunker install)

In [ ]:
import os

import zarr
from rechunker import rechunk

### Open the source data

In [10]:
data_dir = os.environ['DATA_DIR']
geopolar_filepath = os.path.join(data_dir, 'noaa-geopolar-space-chunked.zarr')
zarr.open(geopolar_filepath).analysed_sst.info

Name,/analysed_sst
Type,zarr.core.Array
Data type,float32
Shape,"(124, 3600, 7200)"
Chunk shape,"(2, 3600, 7200)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,12856320000 (12.0G)
No. bytes stored,3506949738 (3.3G)


### Run Rechunker

NOTES:
This didn't work with `'time': -1`, it also didn't work with using just one dictionary `{'time': 124, 'lat': 650, 'lon': 650}`. I understood from the tutorial that it was supposed to broadcast the shape out, but that didn't seem to work for me.
Also, I had to set the memory really high. If I kept it at `'1MB'` I got an error that I ran out of memory.

In [14]:
source_group = zarr.open(geopolar_filepath)
target_chunks = {
    'analysed_sst': {'time': 124, 'lat': 650, 'lon': 650},
    'analysis_error': {'time': 124, 'lat': 650, 'lon': 650},
    'mask': {'time': 124, 'lat': 650, 'lon': 650},
    'sea_ice_fraction': {'time': 124, 'lat': 650, 'lon': 650},
    'time': None, # don't rechunk this array
    'lon': None,
    'lat': None,
	}

max_mem = '400MB'

target_store = os.path.join(data_dir, 'noaa-geopolar-time-chunked.zarr')
temp_store = 'noaa-geopolar-time-chunked-tmp.zarr'

array_plan = rechunk(
    source_group, target_chunks, max_mem, target_store,
    temp_store = temp_store,
    )

Attempting to consolidate the metadata, so I don't get that warning everytime on load. (unsuccessful)

In [39]:
time_chunked = zarr.open(target_store)

zarr.consolidate_metadata(time_chunked.analysed_sst)

AttributeError: 'numpy.ndarray' object has no attribute 'endswith'